<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M3/exercises/CNN_workshop_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages('reticulate')
install.packages('magrittr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
devtools::install_github("rstudio/keras") 

reticulate (NA    -> 6a0e23f94...) [GitHub]
magrittr   (1.5   -> 2.0.1       ) [CRAN]
vctrs      (0.3.4 -> 0.3.5       ) [CRAN]
rappdirs   (NA    -> 0.3.1       ) [CRAN]
rstudioapi (0.12  -> 0.13        ) [CRAN]
config     (NA    -> 0.3         ) [CRAN]
tfruns     (NA    -> 1.4         ) [CRAN]
zeallot    (NA    -> 0.1.0       ) [CRAN]
tensorflow (NA    -> 2.2.0       ) [CRAN]


rappdirs (NA -> 0.3.1) [CRAN]


Installing 1 packages: rappdirs

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✔  checking for file ‘/tmp/Rtmps7SSNu/remotes3938e78b64/rstudio-reticulate-6a0e23f/DESCRIPTION’
─  preparing ‘reticulate’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘reticulate_1.18-9000.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing 8 packages: magrittr, vctrs, rappdirs, rstudioapi, config, tfruns, zeallot, tensorflow

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Skipping install of 'reticulate' from a github remote, the SHA1 (6a0e23f9) has not changed since last install.
  Use `force = TRUE` to force installation



✔  checking for file ‘/tmp/Rtmps7SSNu/remotes392342bfcc/rstudio-keras-aaddf0e/DESCRIPTION’
─  preparing ‘keras’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts (471ms)
─  checking for empty or unneeded directories
   Removed empty directory ‘keras/man-roxygen’
─  building ‘keras_2.3.0.0.9000.tar.gz’
   


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
library(tidyverse)
library(magrittr)
library(keras)

In [5]:
# Let's start by downloading and exploring the data
temp <- tempfile()
download.file('https://storage.googleapis.com/sds-file-transfer/hot-dog-not-hot-dog.zip',temp)
unzip(temp)
unlink(temp)

In [28]:
list.files(path = "test") %>% head()
list.files(path = "train") %>% head()

[1] "hot_dog"     "not_hot_dog"

[1] "hot_dog"     "not_hot_dog"

# Preprocessing

Same as in the former notebook

In [29]:
class_list <- c('hot_dog', 'not_hot_dog')
output_n <- length(class_list)
img_width <- 224
img_height <- 224
target_size <- c(img_width, img_height)
channels <- 3
train_files_path <- 'train'
test_files_path <- 'test'

In [30]:
train_data_gen = image_data_generator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = TRUE)
test_data_gen <- image_data_generator(rescale = 1/255)  

In [31]:
train_image_array_gen <- flow_images_from_directory(train_files_path, train_data_gen, target_size = target_size, class_mode = "binary", 
                                                    classes = class_list,batch_size = 32, seed = 1337)

test_image_array_gen <- flow_images_from_directory(test_files_path, test_data_gen, target_size = target_size, class_mode = "binary",
                                          classes = class_list, batch_size = 32, seed = 1337)

# Loading pretrained model

Keras has some great built-in applications - basically pretrained models with some nice functional overhead. We will start by loadeing VGG16 (a rather large model with many many many layers that has been trained on imagenet)



In [9]:
# create the base pre-trained model
base_model <- application_vgg16(
  include_top = FALSE,
  weights='imagenet',
  input_shape = c(img_height, img_width, channels))

In [11]:
predictions <- base_model$output %>% 
  layer_global_average_pooling_2d() %>% 
  layer_dense(units = 1024, activation = 'relu') %>% 
  layer_dense(units = 1, activation = 'sigmoid')

In [12]:
# this is the model we will train
model <- keras_model(inputs = base_model$input, outputs = predictions)

In [13]:
# first: train only the top layers 
for (layer in base_model$layers)
  layer$trainable <- FALSE

In [15]:
# compile the model (should be done *after* setting layers to non-trainable)
model %>% compile(optimizer = 'adam', 
                  loss = 'binary_crossentropy',
                  metric = 'accuracy')

In [32]:
# train the model on the new data for a few epochs
model %>% fit_generator(
  train_image_array_gen,
  steps_per_epoch = 800,
  epochs = 2, 
  verbose = FALSE,
  )

In [34]:
# let's visualize layer names and layer indices to see how many layers we should freeze:
layers <- base_model$layers
for (i in 1:length(layers))
  cat(i, layers[[i]]$name, "\n")

1 input_1 
2 block1_conv1 
3 block1_conv2 
4 block1_pool 
5 block2_conv1 
6 block2_conv2 
7 block2_pool 
8 block3_conv1 
9 block3_conv2 
10 block3_conv3 
11 block3_pool 
12 block4_conv1 
13 block4_conv2 
14 block4_conv3 
15 block4_pool 
16 block5_conv1 
17 block5_conv2 
18 block5_conv3 
19 block5_pool 


In [35]:
# we chose to train the top 2 blocks, i.e. we will freeze the first layers and unfreeze the rest:
for (i in 1:17)
  layers[[i]]$trainable <- FALSE
for (i in 18:length(layers))
  layers[[i]]$trainable <- TRUE

In [36]:
# we need to recompile the model for these modifications to take effect
model %>% compile(optimizer = 'adam', 
                  loss = 'binary_crossentropy',
                  metric = 'accuracy')

In [37]:
# we train our model again (this time fine-tuning the top 2  blocks
# alongside the top Dense layers
# train the model on the new data for a few epochs
model %>% fit_generator(
  train_image_array_gen,
  steps_per_epoch = 800,
  epochs = 2, 
  verbose = FALSE,
  )

In [38]:
model %>% evaluate_generator(test_image_array_gen, steps=500)

loss  accuracy 
0.5702821 0.6780000

# some own image

![](https://do512family.com/wp-content/uploads/2018/04/18222376_1528395153845930_2934228273602805559_n.jpg)


![](http://myminidoxie.com/wp-content/uploads/2012/11/which-came-first-the-wiener-or-the-dachshund.jpg)


In [50]:
download.file('https://do512family.com/wp-content/uploads/2018/04/18222376_1528395153845930_2934228273602805559_n.jpg',
'wiener1.jpg', mode = 'wb')

download.file('http://myminidoxie.com/wp-content/uploads/2012/11/which-came-first-the-wiener-or-the-dachshund.jpg',
'wiener2.jpg', mode = 'wb')

In [51]:
img <- image_load('wiener2.jpg', target_size = target_size) %>% image_to_array() %>% array_reshape(c(1, dim(.)))

In [52]:
res <- predict(model, img)

In [53]:
ifelse(res == 0, 'hot_dog', 'not_hot_dog')

not_hot_dog
